# Padding & Mask
这里，我们单独讲讲在Encoder和Decoder中的Padding和Mask的作用。这两个在所有LLM中都是必不可少的。简单来讲：
- padding和mask都是输入序列中的一种特殊token，不参与注意力的计算
- Padding是为了保证输入序列具有相同的长度，我们需要对序列添加padding使序列长度相同
- 在预训练过程中，对需要预测的token作mask，在微调中，Decoder中，计算到第$i$个token时，对其之后的token作mask，使注意力机制只关注第$i$个token之前。


## 什么是Padding？
Padding是一个与输入序列形状相同的矩阵，用于指示哪些位置的token是需要参与注意力计算，哪些位置的token则不需要计算，是填充值。
- Padding矩阵中，`0`表示真实值，是需要计算注意力的。
- `1`表示填充值，不需要计算。


## 如何使用Padding
在`Self-Attention`的计算过程中，我们计算`Query`查询和`Key`键的`Dot-product`点积来得到注意力分数。在应用softmax函数之前，我们可以使用`padding`来确保填充位置的注意力分数为一个非常大的负数（例如，乘以-1e9）。这样，当应用softmax函数时，这些位置的权重将接近于零，从而确保模型在其计算中忽略这些填充值。

如下面这个示例：
假设我们有一个长度为4的序列：`[A, B, C, <pad>]`，其中`<pad>`是填充标记。对应的padding mask是：`[0, 0, 0, 1]`。

In [1]:
import torch
attention_scores = torch.tensor([[3, 4, 192, 1],[2, 8, 1,1]]) 
mask = torch.tensor([[0,0,0,1],[0,0, 1,1]])
attention_scores = attention_scores.masked_fill(mask == 1, -1e9)
print(attention_scores)


tensor([[          3,           4,         192, -1000000000],
        [          2,           8, -1000000000, -1000000000]])


In [52]:
import torch

In [76]:
# padding
k_pad_idx = 0
q_pad_idx = 0

# token序列 
# src: 第一句话长度为3，第二句话长度为4， 
# 在这个batch中，batch_length=4，第一句话需要padding一个0 
src_token = torch.tensor([[3, 4, 192, 0],[2, 8, 5, 3]]) 

# trg: 第一句话长度为2，第二句话长度为3
# 在这个batch中，batch_length=3，第一句话需要padding一个0 
trg_token = torch.tensor([[6, 7, 0],[11, 28, 9]])

print("src:", src_token.shape)
print("trg:", trg_token.shape)

len_q, len_k = trg_token.size(1), src_token.size(1)
print("src batch max length:", len_q)
print("trg batch max length:", len_k)

src: torch.Size([2, 4])
trg: torch.Size([2, 3])
src batch max length: 3
trg batch max length: 4


In [ ]:
# embeding
# src = torch.randn(2, 4, 512) # 2个batch， 4个长度， 512维度
# trg = torch.randn(2, 3, 512) # 2个batch， 3个长度， 512维度

# 多头Q
# src encode k : 2个batch， 8个头， 4个长度， 单头64维度
src = torch.randn(2, 8, 4, 64) 

# trg decode Q : 2个batch， 8个头， 3个长度， 单头64维度
trg = torch.randn(2, 8, 3, 64) 

In [94]:
# 为配合多头注意力，需要填充维度
src_mask = src_token.ne(k_pad_idx).unsqueeze(1).unsqueeze(2)
trg_mask = trg_token.ne(q_pad_idx).unsqueeze(1).unsqueeze(3)
print("src_token:\n", src_token)
print("trg_token:\n", trg_token)
print('--------------------------------------------------------')
print("src_mask:\n", src_token.ne(k_pad_idx))
print("trg_mask:\n", trg_token.ne(q_pad_idx))
print('--------------------------------------------------------')
print(f"【src_token】:{src_token.shape}->【src_mask】:",src_mask.shape)
print(f"【trg_token】:{trg_token.shape}->【trg_mask】:",trg_mask.shape)

src_token:
 tensor([[  3,   4, 192,   0],
        [  2,   8,   5,   3]])
trg_token:
 tensor([[8, 2, 0],
        [5, 2, 2]])
--------------------------------------------------------
src_mask:
 tensor([[ True,  True,  True, False],
        [ True,  True,  True,  True]])
trg_mask:
 tensor([[ True,  True, False],
        [ True,  True,  True]])
--------------------------------------------------------
【src_token】:torch.Size([2, 4])->【src_mask】: torch.Size([2, 1, 1, 4])
【trg_token】:torch.Size([2, 3])->【trg_mask】: torch.Size([2, 1, 3, 1])


In [85]:
# 批量处理
src_mask_repeat = src_mask.repeat(1, 1, len_q, 1)
trg_mask_repeat = trg_mask.repeat(1, 1, 1, len_k)
print(f"【src_mask】:{src_mask.shape} -> 【src_mask_repeat】:{src_mask_repeat.shape} ")
print(f"【trg_mask】:{trg_mask.shape} -> 【trg_mask_repeat】:{trg_mask_repeat.shape} ")

【src_mask】:torch.Size([2, 1, 1, 4]) -> 【src_mask_repeat】:torch.Size([2, 1, 3, 4]) 
【trg_mask】:torch.Size([2, 1, 3, 1]) -> 【trg_mask_repeat】:torch.Size([2, 1, 3, 4]) 


In [102]:
# & and 操作符 1&1=1, 1&0=0, 0&1=0, 0&0=0
mask = src_mask_repeat & trg_mask_repeat #[2,1,3,4] & [2,1,3,4] =[2,1,3,4]
print(mask[0][0].shape)
print('------------------------------')
print("src_mask \n",src_mask_repeat[0][0].int())
print("trg_mask \n", trg_mask_repeat[0][0].int())
print('------------------------------')
print("mask \n",mask[0][0].int())
# token序列，q=[6, 7, 0], k=[3, 4, 192, 0]， 0为padding-index
# 第一排 1，1，1，0 对应 6 与 3， 4， 192， 0之间的attention mask
# 第二排 1，1，1，0 对应 7 与 3， 4， 192， 0之间的attention mask
# 第三排 0，0，0，0 对应 0 与 3， 4， 192， 0之间的attention mask

torch.Size([3, 4])
------------------------------
src_mask 
 tensor([[1, 1, 1, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 0]], dtype=torch.int32)
trg_mask 
 tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 0, 0, 0]], dtype=torch.int32)
------------------------------
mask 
 tensor([[1, 1, 1, 0],
        [1, 1, 1, 0],
        [0, 0, 0, 0]], dtype=torch.int32)


In [105]:
# Decode Self Masked 
# mask_decode = 
mask_decode = torch.tril(torch.ones(len_q, len_k)).type(torch.BoolTensor)
print(mask_decode.int())

tensor([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 1, 0]], dtype=torch.int32)


In [103]:
# 计算 encode K， decode Q之间的注意力分数
q = trg
k_t = src.transpose(2,3)

print(xq.shape)
score = xq @ xk_t

score_mask = score.masked_fill(mask == 0, -torch.inf)
# score_mask = score.masked_fill(mask == 0, -10000)
# print(score_mask)
print(score_mask[0,0,:,:])

torch.Size([2, 8, 3, 64])
tensor([[  1.1102,  -3.2859, -12.9273,     -inf],
        [ -0.7410,  -2.2688,  14.2815,     -inf],
        [    -inf,     -inf,     -inf,     -inf]])


In [ ]:
seq 10,   6

mask=zeros(10,10)
mask[:6,:6] = ones.tril(6,6)